In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [88]:
import pandas as pd
import numpy as np

df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [89]:
df.sample(5)

,review,sentiment
44905,Must confess to having seen a few howlers in m...,negative
45637,It is not an easy film to watch - it is over t...,positive
36920,I ran across this movie at the local video sto...,positive
12794,I've seen this movie at theater when it first ...,positive
23763,My Santa Lucia Choir was chosen to be in this ...,positive


In [90]:
df.iloc[0,0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

* Remove html tags
* Remove special characters
*  Converting every thing to lower case
* Removing Stop words
* Stemming

In [91]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [92]:
df['sentiment']=df['sentiment'].replace({'positive':1,"negative":0})
df.head()

/tmp/ipykernel_37/1087332947.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment']=df['sentiment'].replace({'positive':1,"negative":0})


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


**re -> is Python’s regular expression module — it helps you search, match, and clean text patterns.**

In [93]:
# to remove the html tag....

import re
clear=re.compile('<.*?>') ## pattern 

re.sub(clear,'',df.iloc[0].review)

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [94]:
def clear_html(text):
    clear=re.compile('<.*?>') ## pattern 
    return re.sub(clear,'',text)


df['review']=df['review'].apply(clear_html)

df.head(2).review

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. The filming tec...
Name: review, dtype: object

In [95]:
def convert_lower(text):
    return text.lower()


df['review']=df['review'].apply(convert_lower)

df.head(2).review

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. the filming tec...
Name: review, dtype: object

In [96]:
## i.isalnum(): -> It returns True if all characters in the string are alphabets (a–z, A–Z) or numbers (0–9) 
## — and False if there’s any space, punctuation, or special character.
def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x


df['review']=df['review'].apply(remove_special)

df.head(2).review

0    one of the other reviewers has mentioned that ...
1    a wonderful little production  the filming tec...
Name: review, dtype: object

In [97]:
remove_special(' th%e @ classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare. forget pretty pictures painted for mainstream audiences, f')

' th e   classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare  forget pretty pictures painted for mainstream audiences  f'

* **nltk** — the Natural Language Toolkit, a powerful NLP library.

* **stopwords** — a list of common words like “the”, “is”, “in”, “and”, etc., that don’t add much meaning to sentences.

In [98]:
import nltk
from nltk.corpus import stopwords

stop_words=stopwords.words('english')
stop_words[:20]

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been']

In [99]:
def remove_stopwords(text):
    words=text.split()
    filter_words=[word for word in words if word not in stop_words]
    return filter_words

df['review']=df['review'].apply(remove_stopwords)

df.head(2).review

0    [one, reviewers, mentioned, watching, 1, oz, e...
1    [wonderful, little, production, filming, techn...
Name: review, dtype: object

**🧠 What is Stemming?**

Stemming means reducing a word to its root form (stem) —
It doesn’t always produce a valid English word, but it keeps the core meaning.

In [100]:
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
def stemming(text):
    filter=[ps.stem(word) for word in text]
    return filter

df['review']=df['review'].apply(stemming)

df.head(2).review

0    [one, review, mention, watch, 1, oz, episod, h...
1    [wonder, littl, product, film, techniqu, unass...
Name: review, dtype: object

In [101]:
def join_words(text):
    return " ".join(text)

df['review']=df['review'].apply(join_words)

df.head(2).review

0    one review mention watch 1 oz episod hook righ...
1    wonder littl product film techniqu unassum old...
Name: review, dtype: object

In [104]:
X=df['review']
y=df['sentiment']
X

0        one review mention watch 1 oz episod hook righ...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic famili littl boy jake think zombi closet...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job creativ origin fir...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    go disagre previou comment side maltin one sec...
49999    one expect star trek movi high art fan expect ...
Name: review, Length: 50000, dtype: object

**🧠 What is CountVectorizer?**

CountVectorizer converts text (words) into a numeric matrix that machine learning models can understand.

It does this by:

1. Tokenizing text (splitting sentences into words)

2. Building a vocabulary of all unique words

3. Representing each sentence as a vector of word counts

In [118]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=5000) ## Keep only the top 5000 most frequent words in the dataset

X=cv.fit_transform(X).toarray()

## ✅ Each row = 1 review
# ✅ Each column = 1 unique word (feature)

In [119]:
print(cv.get_feature_names_out())

['00' '000' '10' ... 'zombi' 'zone' 'zoom']


In [120]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [121]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [122]:
X_train.shape

(40000, 5000)

In [123]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB

clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()

clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

BernoulliNB()

In [125]:
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)

In [126]:
from sklearn.metrics import accuracy_score

print("Gaussian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernaulli",accuracy_score(y_test,y_pred3))

Gaussian 0.7158
Multinomial 0.8467
Bernaulli 0.8474
